<a href="https://colab.research.google.com/github/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/blob/main/Sentiment_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
#emoji_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5e7d7d29f58be438f33fa0c0bc2fd251dea0b453/emoji.py"
#!rm emoji.py
#!wget $emoji_url
#import emoji

In [3]:
! pip install emoji
import emoji

     |████████████████████████████████| 168 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.0-py3-none-any.whl size=168256 sha256=bfceed326a18320307e9c09506e16af9d13caaabc99d8faed0a99a33b0d85862
  Stored in directory: /root/.cache/pip/wheels/f7/d7/74/c720aaf345a042b0c2d74361873258c5e8649b7f11b2ccce49
Successfully built emoji


In [4]:

emoji.demojize("hello 👍")

'hello :thumbs_up:'

##Szószedet

In [5]:
!rm SampleSubmission*
!rm TEST*
!rm TRAIN*
!rm *zip*
!wget https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
!unzip twitter_sentiment_analysis_ai_challenge-dataset.zip
df=pd.read_csv("TEST.csv")

rm: cannot remove 'SampleSubmission*': No such file or directory
rm: cannot remove 'TEST*': No such file or directory
rm: cannot remove 'TRAIN*': No such file or directory
rm: cannot remove '*zip*': No such file or directory
--2021-10-10 10:25:06--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip [following]
--2021-10-10 10:25:06--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving raw.githubus

In [6]:
df.head()
alltext1=df.text
print (len(alltext1))

2640


In [7]:
df2=pd.read_csv("TRAIN.csv")
alltext2=df2.text
print (len(alltext2))

12000


In [8]:
alltext1=list(alltext1)
alltext2=list(alltext2)
alltext=alltext1+alltext2
print(f" {len(alltext)}")


 14640


In [9]:
!pip install pyspellchecker
from spellchecker import SpellChecker

spell = SpellChecker()
print(spell.correction("@frustrating"))

     |████████████████████████████████| 2.7 MB 5.2 MB/s 
frustrating


In [10]:
import re

def remove_URL(text):
    """Remove URLs from a text string"""
    return re.sub(r"http\S+", "", text)

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def is_number(inp:str):
    a=re.search("[\d]",inp)
    if a==None: 
        return(False)
    return(True)

def strip_all_entities(text):
    import emoji
    import re,string
    entity_prefixes = ['“','@','#',"!","?",":","_"]
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in ['“','@','#',"!","?"]:
                words.append(word)

    return ' '.join(words)

def de_ex(txt):
    txt=txt.lower()
    exwords={"it's":"it is",
    "aren't":"are not",
    "i've":"i have",
    "can't":"can not",
    "didn't":"did not",
    "i'm":"i am",
    "that's":"that is",
    "let's":" let is",
    "she's":"she is",
    "we're":"we are",
    "they've":"they have",
    "fleet's":"fleet is ",
    "there's":"there is",
    "i'll": "i will"
    }
    txt_list=txt.split()
    for w in txt_list:
        if w in exwords:
            txt=txt.replace(w,exwords[w])
    return (txt)



def query_sentence(index):
    print(f"orig: {alltext[index]}")
    spell=SpellChecker()
    inp=alltext[index]
    inp=remove_URL(inp)
    inp=emoji.demojize(inp)
    inp=de_ex(inp)
    print("demojize: ",inp)
    inp=strip_all_entities(inp)
    print("STRIP: ",inp)
    txt=inp.replace(","," ").replace("."," ").replace("!"," _exclamation_ ").replace("#"," ").replace("???","?").replace("??","?").replace("?"," _questionmark_ ").replace("  "," ").replace("  "," ").replace('"',"").replace("“","")
    print("after replace=",txt)
    #print(txt)
    normal=""
    for i in txt.split(" "):
        w1=i.lower().strip()
        if "_" in w1 or ":" in w1:
            w2=w1
        else:
            if w1 !="":
                w2=spell.correction(w1).strip()
            else:
                w2=""
            if w2=="i":
                pass
                #print(f"chek: {w1}, {w2}")
            if is_number(w2):
                w2=""    
        normal += w2+" "
    normal=normal.replace("::",": :")
    print(f"  conv: {normal}")
    return(normal)

##Mentés

In [11]:
t=[]
for i in range(len(alltext)):
    print(f"{i}", end=" ")
    t.append(query_sentence(i))


A streamkimeneten csak az utolsó 5000 sor látható.
  conv: just sent thank u it please do your confirmation number if relight booking problems is needed thank you ey 
13643 orig: @JetBlue Not trying to make you look bad, on your website it says: "Due to weather in the Charleston, NC" Its actually in SC, not NC
demojize:  @jetblue not trying to make you look bad, on your website it says: "due to weather in the charleston, nc" its actually in sc, not nc
STRIP:  not trying to make you look bad on your website it says: due to weather in the charleston nc its actually in sc not nc
after replace= not trying to make you look bad on your website it says: due to weather in the charleston nc its actually in sc not nc
  conv: not trying to make you look bad on your website it says: due to weather in the charleston no its actually in so not no 
13644 orig: @United super lame that you charge $8 for tv on an international flight to the US. It isn’t 1997!
demojize:  @united super lame that you charge

In [49]:
s=set()
for sent in t:
    wlist=sent.strip().split()
    for w in wlist:
        s.add(w)
print (len(s))
ordered=list(s)
ordered.sort()

9627


In [50]:
print(ordered)

['0_0', '10:00p', '10:00pm', '10:01', '10:01pm', '10:05am', '10:05pm', '10:07p', '10:10', '10:14am', '10:15am', '10:20', '10:20pm', '10:30', '10:30a', '10:30pm', '10:35', '10:39', '10:45', '10:45pm', '10:50am', '10:51', '10:51pm', '10:55', '10:55pm', '11:00', '11:00pm', '11:10p', '11:15', '11:15p', '11:20', '11:21p', '11:23', '11:30', '11:30a', '11:30am', '11:30pm', '11:31', '11:35', '11:40', '11:40am', '11:45am', '11:47', '11:50', '11:50pm', '11:51', '11:55am', '11:56', '11:59', '12:00', '12:08', '12:13pm', '12:15', '12:17pm', '12:20', '12:26pm', '12:30', '12:30am', '12:40', '15:30', '1:', '1:00pm', '1:02', '1:05', '1:10', '1:15', '1:23', '1:30', '1:30am', '1:30pm', '1:34', '1:36', '1:40pm', '1:45', '1:45pm', '1pm10:15', '2:', '2:00', '2:00pm', '2:10pm', '2:12', '2:14', '2:15', '2:20', '2:26', '2:30', '2:35pm', '2:45', '2:45hrs', '2:50pm', '2:55', '2:55pm', '2day:', '3:', '3:00am', '3:05pm', '3:15', '3:22', '3:30', '3:30am', '3:30pm', '3:35', '3:50', '3:55', '3:55pm', '3a:right_arrow:

In [51]:
sentence=pd.DataFrame(t)
sentence.columns=["Sentence"]
sentence.to_csv("Sentences_emoji.csv")

In [52]:
wordlist=pd.DataFrame(ordered)
wordlist.columns=["Words"]
wordlist.to_csv("words_emoji.csv")

###---------------------------------


###Adatbetöltés SZAVAK BETÖLTÉSE

In [53]:
import pandas as pd

In [54]:
#words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/words.csv"
words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5b6a8293d5c08614e7997e477df13d6044dfad01/words_emoji_2.csv"
words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5de86fcaa648f3b456cfcc6514b6a045ca095c17/words_emoji_3.csv"


In [55]:
!rm words* 
!wget $words_url

--2021-10-10 11:26:53--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5de86fcaa648f3b456cfcc6514b6a045ca095c17/words_emoji_3.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5de86fcaa648f3b456cfcc6514b6a045ca095c17/words_emoji_3.csv [following]
--2021-10-10 11:26:53--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5de86fcaa648f3b456cfcc6514b6a045ca095c17/words_emoji_3.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123520 (121K) [text/plain]
Saving to: ‘words_emoji_3.csv’

words_emoj

In [56]:
words=pd.read_csv("words_emoji_3.csv",index_col=0)
words.head()
words_list=list(words["Words"])

In [57]:
word_dict={v:i+1 for i,v in enumerate(words_list)}

### adatbetöltés mondatok betöltése

In [59]:
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/Sentences.csv"
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/b9567d54968bba4d03ce8c3cac5ecdc4fe1c6031/Sentences_emoji.csv"
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5b6a8293d5c08614e7997e477df13d6044dfad01/Sentences_emoji_2.csv"
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5de86fcaa648f3b456cfcc6514b6a045ca095c17/Sentences_emoji_3.csv"


In [60]:
!rm Sentences*
!wget $sentences_url
sentences=pd.read_csv("Sentences_emoji_3.csv",index_col=0)
sentences.head()
sentences_list=list(sentences["Sentence"])

--2021-10-10 11:27:08--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5de86fcaa648f3b456cfcc6514b6a045ca095c17/Sentences_emoji_3.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5de86fcaa648f3b456cfcc6514b6a045ca095c17/Sentences_emoji_3.csv [following]
--2021-10-10 11:27:08--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5de86fcaa648f3b456cfcc6514b6a045ca095c17/Sentences_emoji_3.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1470690 (1.4M) [text/plain]
Saving to: ‘Sentences_emoji_3

##Tanulás

In [61]:
!rm *zip*
!rm TEST*
!rm TRAIN*
!rm SampleSubmission*
!wget https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
!unzip twitter_sentiment_analysis_ai_challenge-dataset.zip


--2021-10-10 11:27:18--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip [following]
--2021-10-10 11:27:18--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awa

In [ ]:
df=pd.read_csv("TRAIN.csv")

In [ ]:
df.head(4)

In [ ]:
for i in range(len(df)):
    if "😊" in df.loc[i].text:
        print (f"{i:6}. {df.loc[i].airline_sentiment:9} -- {df.loc[i].retweet_count:5} --  {df.loc[i].text}")

In [ ]:
for i in range(len(df)):
    if df.loc[i].retweet_count>0:
        print (f"{i:6}. {df.loc[i].airline_sentiment:9} -- {df.loc[i].retweet_count:5} --  {df.loc[i].text}")

In [ ]:
sentences_list[10420]

In [ ]:
df.head(1)

In [ ]:
df["clean_text"]=sentences_list[2640:]

In [ ]:
def print_df(df,start=0,stop=0):
    for i in range(start,stop):
        print(f"{i:4}.  --{df.loc[i].text} -- {df.loc[i].clean_text}")

In [ ]:
print_df(df,1,10)

In [ ]:
all_sent=[]
for sentence in df.clean_text:
    sent_list=sentence.strip().split()
    out=[]
    for word in sent_list:
        out.append(word_dict[word]) 
    all_sent.append(out)

In [ ]:
df["sentence_code"]=all_sent

In [ ]:
table=list(df["sentence_code"]) # typo
lens=max([len(x) for x in table])
sentiment=list(df["airline_sentiment"])


In [ ]:
def ylabel_convert(sentiment):
    ytrain=[]
    for sent in sentiment:
        if sent=="negative":
            o=[1,0]
        if sent=="neutral":
            o=[0.5,0.5]
        if sent=="positive":
            o=[0,1]
        ytrain.append(o)
    return(ytrain)

def ylabel_convert3(sentiment):
    ytrain=[]
    for sent in sentiment:
        if sent=="negative":
            o=[1,0,0]
        if sent=="neutral":
            o=[0,1,0]
        if sent=="positive":
            o=[0,0,1]
        ytrain.append(o)
    return(ytrain)

        

In [ ]:
ytrain=ylabel_convert3(sentiment)

In [ ]:
x0=[0 for _ in range(lens) ]
xtrain=[]
for sent in table:
    o1=list(x0[0:lens-len(sent)]+list(sent))
    xtrain.append(o1)

In [ ]:
x_train=xtrain[0:-20]
y_train=ytrain[0:-20]
x_test=xtrain[-20:]
y_test=ytrain[-20:]

In [ ]:


lstm_size=lens
max_input_length=lens
embedding_size=190 #(100: 73%)
n_words=len(table)
n_out=len(y_train[0])

In [ ]:
lens
n_out

In [ ]:

# Importáld a megfelelő rétegeket
from tensorflow.keras.layers import Input,Dense,Embedding,LSTM,TimeDistributed, Flatten, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta,Adam,SGD
from tensorflow.keras.losses import sparse_categorical_crossentropy,categorical_crossentropy, mean_squared_error
from tensorflow.keras import backend as K

K.clear_session
x= Input(shape=(max_input_length,))
embedded_x=Embedding(n_words+1,embedding_size, input_length=max_input_length-1, mask_zero=True)(x)
lstm_output= LSTM(lstm_size,return_sequences=True)(embedded_x)
lstm_output= LSTM(lstm_size,return_sequences=True)(lstm_output)


lstm_output=Flatten()(lstm_output)
#Dense_out= Dense(33, activation="softmax")(lstm_output)

predictions= Dense(n_out, activation="softmax")(lstm_output)
model=Model(inputs=x, outputs=predictions,)



In [ ]:
model.summary()

In [ ]:
x

In [ ]:
import numpy as np

In [ ]:
#train_x = np.asarray(xtrain)
#train_y = np.asarray(ytrain)

In [ ]:
# Loss 

loss = mean_squared_error #categorical_crossentropy # One-hot enkódolt kimenetünk van. Mit is használunk?

# Optimizer
optimizer = Adam() #Ízlés szerint...
 
# Compilation
#############

model.compile(optimizer=optimizer,loss=loss)

In [ ]:
# Illesszük az adatra a modellt.
history=model.fit(x=xtrain,y=ytrain,validation_data=( x_test,y_test), epochs=6, batch_size=50,)


In [ ]:
pred=model.predict(x_train)

In [ ]:
for i in range(len(pred)):
    print(f"{i}, {pred[i]}, {y_train[i]}")



### Model usage

In [ ]:
test_df=pd.read_csv("TEST.csv")

In [ ]:
print(len(test_df))

In [ ]:
test_df.head(130)

In [ ]:
test_df["clean_text"]=sentences_list[:2640]

In [ ]:
def code_sentences(df):
    all_sent=[]
    for sentence in df.clean_text:
        sent_list=sentence.strip().split()
        out=[]
        for word in sent_list:
            out.append(word_dict[word]) 
        all_sent.append(out)
    df["sentence_code"]=all_sent
    return (df)

In [ ]:
test_df=code_sentences(test_df)

In [ ]:
def push_sentences(df,lens=33):
    table=list(df["sentence_code"]) # typo
    
    print(lens)
    x0=[0 for _ in range(lens) ]
    x=[]
    for sent in table:
        o1=list(x0[0:lens-len(sent)]+list(sent))
        x.append(o1)
    return(x)

In [ ]:
test_df.head(1)

In [ ]:

xx=push_sentences(test_df,lens=59)

In [ ]:
test_out=model.predict(xx)

In [ ]:
test_out

In [ ]:
def decode3(model_out):
    y=[]
    for mo in model_out:
        if mo[0]>mo[1] and mo[0]>mo[2]:
            o="negative"    
        if mo[1]>mo[0] and mo[1]>mo[2]:
            o="neutral"    
        if mo[2]>mo[0] and mo[2]>mo[1]:
            o="positive"    
        y.append(o)
    return(y)


  

In [ ]:
def decode(model_out):
    y=[]
    for mo in model_out:
        dif=abs(mo[0]-mo[1])
        if dif < 0.2 :
            o="neutral"
        else:
            if mo[0]>mo[1] :
                o="negative"    
            else:
                o="positive"    
        y.append(o)
    return(y)



In [ ]:
prediction=decode3(test_out)

In [ ]:
def listing(status):
    counti=0
    for i in range(len(prediction)):
    
        predi=prediction[i]
        reason=test_df.loc[i].negativereason
        airline_sentiment=test_df.loc[i].airline_sentiment_gold
        if predi==status:
            counti+=1
            print(f"{i:4}. {counti:3}. =  {prediction[i]} -- {reason:15}--{airline_sentiment} -- {test_df.loc[i].text}")
            cond=[reason=="Flight Booking Problems", 
                    reason=="Bad Flight", 
                    reason=="Late Flight", 
                    reason=="Lost Luggage", 
                    reason=="Customer Service Issue",
                    reason=="Cancelled Flight",
                    reason=="Flight Attendant Complaints",
                    reason=="longlines",
                    reason=="Damaged Luggage"
                    ]
            if any(cond):
                prediction[i]="negative"


In [ ]:
df.head(1)

In [ ]:
def retweet_converter():
    counti=0
    for i in range(len(prediction)):
    
        predi=prediction[i]
        reason=test_df.loc[i].negativereason
        retweet=test_df.loc[i].retweet_count
        if retweet>0:
            prediction[i]="negative"

In [ ]:
listing("neutral")
#retweet_converter()

In [ ]:
test_df["predicted"]=prediction

In [ ]:
prediction_df=pd.DataFrame(prediction)

In [ ]:
prediction_df.columns=["Sentiment"]

In [ ]:
prediction_df.head()

In [ ]:
prediction_df.index.name="Index"

In [ ]:
from datetime  import datetime

In [ ]:
a=datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
print(a)

In [ ]:
prediction_df.to_csv("submission_"+a+".csv")

In [ ]:
predcsv=test_df.loc[:,["text","predicted"]]

predcsv.to_csv("mini.csv")

In [ ]:
print_df(test_df,95,145)

In [ ]:
def de_ex(txt):
    txt=txt.lower()
    exwords={"it's":"it is",
    "aren't":"are not",
    "i've":"i have",
    "can't":"can not",
    "didn't":"did not",
    "i'm":"i am",
    "that's":"that is",
    "let's":" let is",
    "she's":"she is",
    "we're":"we are",
    "they've":"they have",
    "fleet's":"fleet is ",
    "there's":"there is"
    "i'll": "i will"
    }
    txt_list=txt.split()
    for w in txt_list:
        if w in exwords:
            txt=txt.replace(w,exwords[w])
    return (txt)


In [ ]:
de_ex("hello i'm bond, james bond")

In [ ]:
def print_char_df(df,character,start=0,stop=0):
    for i in range(start,stop):
        if character in df.loc[i].text:
            print(f"{i:4}.  --{df.loc[i].text} -- {df.loc[i].clean_text}")

In [ ]:
print_char_df(test_df,"'",0,100)